<a href="https://colab.research.google.com/github/MoAI-Dev-Team/MoAI/blob/master/(int%2Cint)_%3Eint_10_%E5%AE%8C%E5%85%A8%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext cython

In [ ]:
%%cython
import cython
import collections
from random import randint
import sys

sys.setrecursionlimit(1000000)


def flatten(list l):
  for el in l:
    if isinstance(el, list):
      yield from flatten(el)
    else:
      yield el

def abs_num(list x,list y):
  j = 0
  while True:
    if x[j] == 'λ':
      y[j:] = list(map(lambda x:x+1, y[j:]))
      j += 1
    elif x[j] == "@":
      pk = j+1
      pl = 1
      #代入される対象
      while pl != 0:
        if x[pk] == "@":
          pl += 1
        if isinstance(x[pk], int) or (x[pk] == 'x') or (x[pk] == 'y'):
          pl -= 1
        pk += 1
      ps = pk

      #代入する対象
      pl = 1
      while pl != 0:
        if x[pk] == "@":
          pl += 1
        if isinstance(x[pk], int) or (x[pk] == 'x') or (x[pk] == 'y'):
          pl -= 1
        pk += 1
  
      y = y[:j+1] + abs_num(x[j+1:ps],y[j+1:ps]) + abs_num(x[ps:pk],y[ps:pk])
      return y
    else:
      return y   

def abs_num2(list x,list y):
  j = 0
  while True:
    if x[j] != 'x' and x[j] != 'y' and x[j] < 0:
      y[j:] = list(map(lambda x:x+1, y[j:]))
      j += 1
    elif x[j] == 0:
      pk = j+1
      pl = 1
      #代入される対象
      while pl != 0:
        if x[pk] == 0:
          pl += 1
        if x[pk] == 'x' or x[pk] == 'y' or x[pk] > 0:
          pl -= 1
        pk += 1
      ps = pk

      #代入する対象
      pl = 1
      while pl != 0:
        if x[pk] == 0:
          pl += 1
        if x[pk] == 'x' or x[pk] == 'y' or x[pk] > 0:
          pl -= 1
        pk += 1
      y = y[:j+1] + abs_num2(x[j+1:ps],y[j+1:ps]) + abs_num2(x[ps:pk],y[ps:pk])
      return y
    else:
      return y

cpdef to_reduction(list x):
  if x == []:
    return []  
  y = [0]*len(x)
  t = abs_num(x,y)
  u = []
  for i in range(len(x)):
    if x[i] == "λ":
      #λマイナス
      u.append(-t[i])
    elif isinstance(x[i], int):
      u.append(t[i]-x[i]+1)
    elif (x[i] == 'x') or (x[i] == 'y'):
      u.append(x[i])
    else:
      #at0
      u.append(0)

  #α変換
  k = []
  for i in range(len(x)):
    if isinstance(u[i],int) and u[i] < 0:
      if u[i] in k:
        #木構造を探す
        # +1
        pk = i+1
        pl = 1
        while pl != 0:
          if u[pk] == 0:
            pl += 1
          elif isinstance(u[pk],list) or u[pk] > 0:
            pl -= 1
          pk += 1
        s = min(k)-1
        for j in range(i+1,pk):
          if u[j] == -u[i]:
            u[j] = [-s]            
        u[i] = [s]
        k.append(s)
      else:  
        k.append(u[i])
  u = list(flatten(u))                 
  return u

cpdef de_brujin(list x): 
  g = 0
  j = 0
  while j != len(x):
    if (x[j] != 'x') and (x[j] != 'y') and x[j] == 0 and (x[j+1] != 'x') and (x[j+1] != 'y') and x[j+1] < 0:
      pk = j+1
      pl = 1
      #代入される対象
      while pl != 0:
        if x[pk] == 0:
          pl += 1
        if (x[pk] == 'x') or (x[pk] == 'y') or (x[pk] > 0):
          pl -= 1
        pk += 1
      ps = pk
      #代入する対象
      pl = 1
      while pl != 0:
        if x[pk] == 0:
          pl += 1
        if (x[pk] == 'x') or (x[pk] == 'y') or (x[pk] > 0):
          pl -= 1
        pk += 1
      k = 0
      for l in range(j+2,ps):  
        if (x[l] != 'x') and (x[l] != 'y') and (x[l] + x[j+1] == 0):
          x[l] = x[ps:pk]

      del x[ps:pk]
      del x[j]
      del x[j]
      x = list(flatten(x))
  
      #アルファ変換
      k = min(x)  
      for i in range(len(x)):
        if x[i] < 0:
          if x[i] in x[:i]:
            #木構造を探す
            # +1
            pk = i+1
            pl = 1
            while pl != 0:
              if x[pk] == 0:
                pl += 1
              if x[pk] > 0:
                pl -= 1
              pk += 1
            s = k-1
            for j in range(i+1,pk):
              if x[j] == -x[i]:
                x[j] = -s
            x[i] = s
            k = s   
      g += 1
      j = 0
      if g == 100 or len(x) > 1000:
        return []
    else:            
      j += 1
  return x  

cpdef to_de_brujin(list x):
  if x == []:
    return []
  u = []
  y = abs_num2(x,[0]*len(x))
  for i in range(len(x)):
    if x[i] != 'x' and x[i] != 'y' and x[i] < 0:
      u.append([-x[i],y[i]])
      x[i] = -y[i]
    
  for i in range(len(u)):
    x = [[u[i][1]] if j == u[i][0] else j for j in x]
  x = list(flatten(x))
  
  for i in range(len(x)):
    if x[i] == 0:
      x[i] = "@"
    elif x[i] != 'x' and x[i] != 'y' and x[i] < 0:
      x[i] = "λ"
    elif x[i] == 'x' or x[i] == 'y':
      pass
    else:
      x[i] = y[i]-x[i] + 1  
  return x    

cpdef reduction(list x): #簡約
  x = to_reduction(x) #＠とλを数字に変える
  x = de_brujin(x)   #実際の簡約
  x = to_de_brujin(x) #＠とλに戻す
  return x

cpdef closed1(list f): #組み合わせで使う
  s = abs_num(f, [0]*len(f))
  for i in range(len(f)):
    if f[i] == 'x':
      f[i] = s[i] + 1
    elif f[i] == 'y':
      f[i] = s[i] + 2
  f = ['λ','λ'] + f 
  return f    

cpdef closed2(list f): #組み合わせで使う
  s = abs_num(f, [0]*len(f))
  for i in range(len(f)):
    if f[i] == 'x':
      f[i] = s[i] + 2
    elif f[i] == 'y':
      f[i] = s[i] + 1
  f = ['λ','λ'] + f 
  return f  

cpdef church(int s):
  if s == 1:
    return ["λ",1]
  else:  
    return ["λ","λ"]+["@",2]*s+[1] #ここはド・ブラウン・インデックスかレベルかで変わるから注意

cpdef combine(list f,list g): #組み合わせ
  q = randint(1,12)
  if q == 1:
    s = ['@','@']+f+['@','@']+g+church(randint(0,9))+['x']+['y']
  if q == 2:
    s = ['@','@']+f+['@','@']+g+['x']+church(randint(0,9))+['y']
  if q == 3:  
    s = ['@','@']+f+['@','@']+g+['x']+['y']+church(randint(0,9))
  if q == 4:
    s = ['@','@']+f+church(randint(0,9))+['@','@']+g+['x']+['y']
  if q == 5:
    s = ['@','@']+f+['x']+['@','@']+g+church(randint(0,9))+['y']
  if q == 6:
    s = ['@','@']+f+['x']+['@','@']+g+['y']+church(randint(0,9))
  if q == 7:
    s = ['@','@']+f+['@','@']+g+['x']+['x']+['y']
  if q == 8:
    s = ['@','@']+f+['@','@']+g+['x']+['y']+['x']
  if q == 9:
    s = ['@','@']+f+['@','@']+g+['y']+['x']+['x']  
  if q == 10:
    s = ['@','@']+f+['x']+['@','@']+g+['x']+['y']
  if q == 11:
    s = ['@','@']+f+['x']+['@','@']+g+['y']+['x']
  if q == 12:
    s = ['@','@']+f+['y']+['@','@']+g+['x']+['x']      
  return s  

In [ ]:
from random import random,uniform,randint,shuffle,sample,choice
import numpy as np
from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Dense, Input, Activation
from keras.models import Model
import math
import tensorflow as tf
import tensorflow.keras
from sklearn.metrics import confusion_matrix
from keras.models import load_model
import copy

#-------------------------------------------------------------
#generator

def generate_random(upper,app_prob,var_prob):
  i = 1
  generate = ["λ"]
  w = random()
  while len(generate) <= upper:
    if w < app_prob:
      generate.append("@")
      i += 1
      w = uniform(0,app_prob+var_prob)
    elif w < app_prob+var_prob:
      generate.append(0)
      w = random()
      i -= 1
    else:
      generate.append("λ")
      w = random()
    if i == 0:
      t = abs_num(generate,[0]*len(generate))

      for j in range(len(generate)):
        if isinstance(generate[j], int):
          if t[j] == 0:
            i = 1
            generate = ["λ"]
            w = random()
            break
          else:  
            generate[j] = randint(1,t[j])   
      is_eta = to_reduction(generate)
      if eta(is_eta) == True:
        return generate
      else:
        i = 1
        generate = ["λ"]
        w = random()   
    if len(generate) == upper:
      i = 1
      generate = ["λ"]
      w = random()

def eta(x): 
  j = 0
  while j <= len(x)-1:
    if (x[j] < 0) and (x[j+1] == 0):
      pk = j+2
      pl = 1
      #代入される対象
      while pl != 0:
        if x[pk] == 0: #@
          pl += 1
        if x[pk] > 0: #e
          if x[j] + x[pk] == 0:
            j += 1
            break
          pl -= 1
        pk += 1     
      if x[j] + x[pk] == 0:
        return False  
      else:
        j += 1
    else:
      j += 1      
  return True

def generate_random_mutate(upper,app_prob,var_prob,body_abs_num):
  i = 1
  generate = []
  w = random()
  while len(generate) <= upper:
    if w < app_prob:
      generate.append("@")
      i += 1
      w = uniform(0,app_prob+var_prob)
    elif w < app_prob+var_prob:
      generate.append(0)
      w = random()
      i -= 1
    else:
      generate.append("λ")
      w = random()
    if i == 0:
      t = abs_num(generate,[0]*len(generate))

      for j in range(len(generate)):
        if isinstance(generate[j], int):
          generate[j] = randint(1,t[j]+body_abs_num)  
      return generate      
    if len(generate) == upper:
      i = 1
      generate = []
      w = random()

#-------------------------------------------------------------
upper = 30
app_prob = 0.3
var_prob = 0.4
POP_SIZE = 100
GENERATIONS = 500
MAX_ANSWER = 10

def init_population():
  pop = []
  while True:
    t = generate_random(upper,app_prob,var_prob)
    if t not in pop:
      pop.append(t)
    if len(pop) >= POP_SIZE:
      break
  return pop

def to_array(x):
  x1 = []
  #最初がラムダだから　自由変数なしにおいては
  for i in range(1,len(x)):
    if x[i] == "@":
      x1 = x1 + [1,0,0]
    elif x[i] == "λ":
      x1 = x1 + [0,1,0]
    else:
      x1 = x1 + [0,0,x[i]]
  x1 = x1 + [0,0,0]*(upper-len(x))
  return np.array(x1)

def ischurch(x):
  if x == church(1):
    return 1
  if x == []:
    return "notchurch"  
  if (x[0] == 'λ') and (x[1] == 'λ'):
    s = 2
    if x[2] == 1:
      return 0    
    while (x[s] == "@") and (x[s+1] == 2):
      if x[s+2] == 1:
        return int(s/2)%10 
      else:
        s += 2
    return "notchurch"    
  else:
    return "notchurch"   

def calc(p0, chosen):
  chosens = []
  individual = np.empty((0,107),int)
  answers = np.empty((0,1),int)
  for i in range(len(chosen)):
    t0 = p0[chosen[i][0]]
    t1 = church(chosen[i][1])
    t2 = church(chosen[i][2])
    t_ans = t0 + t1 + t2 
    t_ans.insert(0,"@")
    t_ans.insert(0,"@")
    individuals = to_array(t0)
    k1,k2 = [0]*10,[0]*10
    k1[chosen[i][1]] = 1
    k2[chosen[i][2]] = 1
    individuals = np.append(individuals,k1)
    individuals = np.append(individuals,k2)
    t_ans_reduced = reduction(t_ans)
    t = ischurch(t_ans_reduced)
    if isinstance(t,int):
      answers = np.append(answers, np.array([[t]]))
    else: 
      answers = np.append(answers, np.array([[MAX_ANSWER]]))   
    individual = np.append(individual, [individuals], axis=0)
    chosens.append(chosen[i])  
  return individual, answers, chosens

def evaluation(p0, train, test, generation):

  individuals1, answers1, chosen1 = calc(p0, train)
  individuals2, answers2, chosen2 = calc(p0, test)
  
  x_train = np.array(individuals1, np.float32)
  y_train = answers1
  y_train = to_categorical(y_train, MAX_ANSWER+1)

  x_test = np.array(individuals2, np.float32)
  y_test1 = answers2
  y_test = to_categorical(y_test1, MAX_ANSWER+1)
  
  crossentropy = learner(x_train, y_train, x_test, y_test, y_test1, generation == 0)  
  
  pop_evaluate = []
  for l in range(POP_SIZE):
    popk_evaluate = []
    for k in range(len(crossentropy)):
      if chosen2[k][0] == l:
        popk_evaluate.append(crossentropy[k])
      
    popk_evaluate_ave = sum(popk_evaluate) / len(popk_evaluate)   
    pop_evaluate.append(popk_evaluate_ave)
  
  return pop_evaluate
  

class Mish(Activation):
    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'

def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

get_custom_objects().update({'Mish': Mish(mish)})

def learner(x_train, y_train, x_test, y_test, y_test1, flag):
  inputs = Input(shape=(107,))
  x = Dense(100, activation='Mish')(inputs)
  x = Dense(100, activation="Mish")(x)
  x = Dense(MAX_ANSWER+1, activation="softmax")(x)


  model = Model(inputs=inputs, outputs=x)          

  model.compile(loss='categorical_crossentropy',optimizer='adamax',metrics=['accuracy']) #adamaxの中身もいじる？？？
  if flag:
    pass
  else:  
    model.load_weights('transfer0.h5')              
  model.fit(x_train, y_train, epochs=1, batch_size=1, validation_data=(x_test, y_test))
  model.save_weights('transfer0.h5')
  predict_prob=model.predict(x_test)
  predict_classes=np.argmax(predict_prob,axis=1)
  true_classes = y_test1
  #print(confusion_matrix(true_classes, predict_classes))
#予測 
  crossentropy = []
  
  w = model.predict(x_test)
  for i in range(num_test_times*POP_SIZE):
    if y_test[i][0] == 1:
      crossentropy.append(w[i][0])  
    elif y_test[i][1] == 1:
      crossentropy.append(w[i][1])
    elif y_test[i][2] == 1:
      crossentropy.append(w[i][2])  
    elif y_test[i][3] == 1:
      crossentropy.append(w[i][3])  
    elif y_test[i][4] == 1:
      crossentropy.append(w[i][4])  
    elif y_test[i][5] == 1:
      crossentropy.append(w[i][5])  
    elif y_test[i][6] == 1:
      crossentropy.append(w[i][6])  
    elif y_test[i][7] == 1:
      crossentropy.append(w[i][7])  
    elif y_test[i][8] == 1:
      crossentropy.append(w[i][8])  
    elif y_test[i][9] == 1:
      crossentropy.append(w[i][9])                  
    else:   
      crossentropy.append(1)
  return crossentropy
  
eli_rate=0.1 
mut_rate=0.2
sel_rate=0.2 
com_rate=0.5 
tournament_size=1 

def new_generation(population, fitnesses):

  popfit_old = []
  for i in range(POP_SIZE):
    popfit_old.append([fitnesses[i]]+[population[i]])

  popfit_old.sort(key=lambda x: x[0])

  print(popfit_old[0][1])

  pop_old = []
  
  for j in range(POP_SIZE):
    pop_old.append(popfit_old[j][1])

  #elite
  new = pop_old[:int(POP_SIZE*eli_rate)]

  tournament_list = list(range(POP_SIZE)) 

  #selection
  for j in range(int(POP_SIZE*sel_rate/2)):
    while True:
      pop0_selected = pop_old[min(sample(tournament_list, tournament_size))]
      pop1_selected = pop_old[min(sample(tournament_list, tournament_size))]     
      x, y = selection(pop0_selected.copy(),pop1_selected.copy())
      if (1 <= len(x) <= upper) and (1 <= len(y) <= upper) and (x not in new) and (y not in new): 
        is_etax = to_reduction(x)
        is_etay = to_reduction(y)
        if (eta(is_etax) == True) and (eta(is_etay) == True):
          new.append(x)
          new.append(y)         
          break    

  #combination
  for j in range(int(POP_SIZE*com_rate/2)):
    while True:
      pop0_selected = pop_old[min(sample(tournament_list, tournament_size))]
      pop1_selected = pop_old[min(sample(tournament_list, tournament_size))]     
      x = combine(pop0_selected,pop1_selected)
      x = closed1(x)
      x = reduction(x)
      if (1 <= len(x) <= upper) and (x not in new): 
        is_eta = to_reduction(x)
        if eta(is_eta) == True:        
          new.append(x)          
          break

  for j in range(int(POP_SIZE*com_rate/2)):
    while True:
      pop0_selected = pop_old[min(sample(tournament_list, tournament_size))]
      pop1_selected = pop_old[min(sample(tournament_list, tournament_size))]     
      x = combine(pop0_selected,pop1_selected)
      x = closed2(x)
      x = reduction(x)
      if (1 <= len(x) <= upper) and (x not in new): 
        is_eta = to_reduction(x)
        if eta(is_eta) == True:        
          new.append(x)           
          break
                                        

  #mutation
  for i in range(int(POP_SIZE*mut_rate)):
    while True:
      pop_selected = pop_old[min(sample(tournament_list, tournament_size))]
      x = mutation(pop_selected.copy())
      if (1 <= len(x) <= upper) and (x not in new):
        is_eta = to_reduction(x)
        if eta(is_eta) == True:
          new.append(x)            
          break
  return new    
  

def mutation(x): #改良
  int_num = [i for i, s in enumerate(x) if isinstance(s, int)]
  s = choice(int_num)

  body_abs_num = abs_num(x,[0]*len(x))[s]
  x[s] = generate_random_mutate(10,app_prob, var_prob,body_abs_num)
  x = list(flatten(x))
  x = reduction(x)
  return x


def selection(x, y): #改良 
  app_num = [i for i, s in enumerate(x) if s == "@"]
  if 1 >= len(app_num):
    x = []
    return x, y
  else:
    s = choice(app_num)
    pk0 = s+1
    pl = 2
    #代入される対象
    while pl != 0:
      if x[pk0] == "@":
        pl += 1
      if isinstance(x[pk0], int):
        pl -= 1
      pk0 += 1
    i = 0
    while x[s-i-1] == "λ": 
      i += 1
    ex0 = s-randint(0,i)
    app_num = [i for i, s in enumerate(y) if s == "@"]
    if 1 >= len(app_num):
      y = []
      return x, y
    else:
      s = choice(app_num)
      pk1 = s+1
      pl = 2
      #代入される対象
      while pl != 0:
        if y[pk1] == "@":
          pl += 1
        if isinstance(y[pk1], int):
          pl -= 1
        pk1 += 1 
      i = 0
      while y[s-i-1] == "λ":
        i += 1
      ex1 = s-randint(0,i)

      t0,t1 = x[ex0:pk0], y[ex1:pk1]
      xab = abs_num(x,[0]*len(x))[ex0]
      yab = abs_num(y,[0]*len(y))[ex1]
      p = randint(0,abs(xab-yab))
      q = randint(0,abs(xab-yab))

      if xab > yab:
        for i in range(len(t0)):
          if isinstance(t0[i],int):
            t0[i] -= p
        for i in range(len(t1)):
          if isinstance(t1[i],int):
            t1[i] += q
      elif xab < yab:
        for i in range(len(t0)):
          if isinstance(t0[i],int):
            t0[i] += p
        for i in range(len(t1)):
          if isinstance(t1[i],int):
            t1[i] -= q               
   
      x[ex0:pk0],y[ex1:pk1] = t1,t0
      for i in range(len(x)): 
        if isinstance(x[i],int) and ((x[i] > abs_num(x,[0]*len(x))[i]) or (x[i] <= 0)):
          x = []
          break
      for i in range(len(y)):
        if isinstance(y[i],int) and ((y[i] > abs_num(y,[0]*len(y))[i]) or (y[i] <= 0)):
          y = []
          break

      x,y = reduction(x),reduction(y)
      return x,y

import time

def fitness(p0):
  for i in range(GENERATIONS):
    print("generation", i+1)
    #t0 = time.time()
    chosen = make_individuals()
    train,test = chosen[0],chosen[1]

    fits = evaluation(p0, train, test, i)
    p0 = new_generation(p0, fits)       
    #print(time.time()-t0)
    print("-------------------------------------------")    

import itertools

def make_individuals():

  p = itertools.product(range(100),range(10),range(10))
  p = list(p)
  shuffle(p)
  chosen = (p[:num_train_times*POP_SIZE],p[num_train_times*POP_SIZE:])
  return chosen

num_train_times = 50
num_test_times = 50

def main():
  pop0 = init_population()
  fitness(pop0)

if __name__ == '__main__':
  main()

generation 1
5000/5000 [==============================] - 23s 2ms/step - loss: 1.0866 - accuracy: 0.7198 - val_loss: 0.8363 - val_accuracy: 0.7300
['λ', '@', 1, '@', 1, 'λ', 2]
-------------------------------------------
generation 2
5000/5000 [==============================] - 11s 2ms/step - loss: 1.1969 - accuracy: 0.6763 - val_loss: 0.8267 - val_accuracy: 0.7544
['λ', '@', '@', 1, 'λ', 'λ', 3, 'λ', 1]
-------------------------------------------
generation 3
5000/5000 [==============================] - 12s 2ms/step - loss: 1.2665 - accuracy: 0.6411 - val_loss: 0.8828 - val_accuracy: 0.7340
['λ', '@', 1, 'λ', 2]
-------------------------------------------
generation 4
5000/5000 [==============================] - 11s 2ms/step - loss: 1.2468 - accuracy: 0.6514 - val_loss: 0.9062 - val_accuracy: 0.7126
['λ', '@', 1, '@', 1, 'λ', 2]
-------------------------------------------
generation 5
5000/5000 [==============================] - 11s 2ms/step - loss: 1.3740 - accuracy: 0.6109 - val_los